## Notes:
- new data (RTP 2022) is from the remm se output
- old data (RTP 2019) from wfrc data portal

## How employment is calculated:

**tdm_output['ALLEMP']** = tdm_output['RETL'] + tdm_output['FOOD'] + tdm_output['MANU'] + tdm_output['WSLE'] + tdm_output['OFFI'] + tdm_output['GVED'] + tdm_output['HLTH'] + tdm_output['OTHR'] + tdm_output['FM_AGRI'] + tdm_output['FM_MING'] + tdm_output['FM_CONS'] + tdm_output['HBJ']  
    
**tdm_output['RETEMP']** = tdm_output['RETL'] + tdm_output['FOOD']  
**tdm_output['INDEMP']** = tdm_output['MANU'] + tdm_output['WSLE']  
**tdm_output['OTHEMP']** = tdm_output['OFFI'] + tdm_output['GVED']+ tdm_output['HLTH'] + tdm_output['OTHR']  
**tdm_output['TOTEMP']** = tdm_output['RETEMP'] + tdm_output['INDEMP']+ tdm_output['OTHEMP']  

In [1]:
import arcpy
from arcpy import env
import os
import glob
from arcgis import GIS
from arcgis.geometry import Geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import pandas as pd
import zipfile
from datetime import date
import shutil

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [2]:
# import datasets
taz832 = r".\Inputs\TAZ_832_ID_Only.shp"
taz900 = r".\Inputs\TAZ_900.gdb\TAZ_UTM12"
city_areas = r".\Inputs\TAZ_900.gdb\CITYAREA_UTM12"
other_areas = r".\Inputs\TAZ_900.gdb\OTHERAREA_UTM12"
med_districts = r".\Inputs\TAZ_900.gdb\DISTMED_UTM12"
counties = r".\Inputs\TAZ_900.gdb\COUNTY_UTM12"
# small_districts = ???


taz832_sdf = pd.DataFrame.spatial.from_featureclass(taz832)
taz900_sdf = pd.DataFrame.spatial.from_featureclass(taz900)[['SA_TAZID', 'DISTMED', 'CityArea', 'OtherArea', 'CO_NAME', 'DEVACRES','SHAPE']].copy()
city_areas_sdf = pd.DataFrame.spatial.from_featureclass(city_areas)[['CityArea', 'CO_NAME', 'DEVACRES','SHAPE']].copy()
other_areas_sdf = pd.DataFrame.spatial.from_featureclass(other_areas)[['OtherArea', 'CO_NAME', 'DEVACRES','SHAPE']].copy()
med_districts_sdf = pd.DataFrame.spatial.from_featureclass(med_districts)[['DISTMED', 'CO_NAME', 'DEVACRES','SHAPE']].copy()
counties_sdf = pd.DataFrame.spatial.from_featureclass(counties)[['CO_NAME', 'DEVACRES','SHAPE']].copy()

In [3]:
# store the paths to the new SE outputs
new_se_folder = r".\New_SE_Data"
new_taz_se = glob.glob(os.path.join(new_se_folder,'taz900_SE_*.csv'))
# new_taz_se = glob.glob(os.path.join(new_se_folder,'SE_WF_*.csv'))
len(new_taz_se)

32

In [4]:
# store the paths to the old SE outputs
old_se_folder = r".\Old_SE_Data"
old_taz_se = glob.glob(os.path.join(old_se_folder,'*_TAZ.csv'))
old_city_se = glob.glob(os.path.join(old_se_folder,'*_City_Area.csv'))

In [5]:
outputs = r'.\Outputs'
if not os.path.exists(outputs):
    os.makedirs(outputs)

scratch = os.path.join(outputs, 'scratch.gdb')
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, 'scratch.gdb')

### Process New SE Data

In [6]:
base = taz900_sdf.copy()
base.shape

for csv in new_taz_se:

    year = os.path.splitext(os.path.basename(csv))[0].split('_')[-1]
    df = pd.read_csv(csv)
    df['TPCL'] = df['TOTEMP']
    df['HJI'] = df['TOTHH']*1.8 + df['TPCL']
    df = df[[';TAZID', 'TOTHH', 'HHPOP', 'ALLEMP','RETEMP', 'INDEMP', 'OTHEMP', 'TPCL', 'HJI']].copy()
    df.columns = ['SA_TAZID', f'HH_{year}', f'POP_{year}', f'AEMP_{year}', f'RTL_{year}', f'IND_{year}', f'OTHR_{year}', f'TPCL_{year}', f'HJI_{year}']
    df = df[df['SA_TAZID']!= 2209]
    base = base.copy().merge(df.copy(), on='SA_TAZID', how='left')

new_se = base
new_se

,SA_TAZID,DISTMED,CityArea,OtherArea,CO_NAME,DEVACRES,SHAPE,HH_2019,POP_2019,AEMP_2019,RTL_2019,IND_2019,OTHR_2019,TPCL_2019,HJI_2019,HH_2020,POP_2020,AEMP_2020,RTL_2020,IND_2020,OTHR_2020,TPCL_2020,HJI_2020,HH_2021,POP_2021,AEMP_2021,RTL_2021,IND_2021,OTHR_2021,TPCL_2021,HJI_2021,HH_2022,POP_2022,AEMP_2022,RTL_2022,IND_2022,OTHR_2022,TPCL_2022,HJI_2022,HH_2023,POP_2023,AEMP_2023,RTL_2023,IND_2023,OTHR_2023,TPCL_2023,HJI_2023,HH_2024,POP_2024,AEMP_2024,RTL_2024,IND_2024,OTHR_2024,TPCL_2024,HJI_2024,HH_2025,POP_2025,AEMP_2025,RTL_2025,IND_2025,OTHR_2025,TPCL_2025,HJI_2025,HH_2026,POP_2026,AEMP_2026,RTL_2026,IND_2026,OTHR_2026,TPCL_2026,HJI_2026,HH_2027,POP_2027,AEMP_2027,RTL_2027,IND_2027,OTHR_2027,TPCL_2027,HJI_2027,HH_2028,POP_2028,AEMP_2028,RTL_2028,IND_2028,OTHR_2028,TPCL_2028,HJI_2028,HH_2029,POP_2029,AEMP_2029,RTL_2029,IND_2029,OTHR_2029,TPCL_2029,HJI_2029,HH_2030,POP_2030,AEMP_2030,RTL_2030,IND_2030,OTHR_2030,TPCL_2030,HJI_2030,HH_2031,POP_2031,AEMP_2031,RTL_2031,IND_2031,OTHR_2031,TPCL_2031,HJI_2031,HH_2032,POP_2032,AEMP_2032,RTL_2032,IND_2032,OTHR_2032,TPCL_2032,HJI_2032,HH_2033,POP_2033,AEMP_2033,RTL_2033,IND_2033,OTHR_2033,TPCL_2033,HJI_2033,HH_2034,POP_2034,AEMP_2034,RTL_2034,IND_2034,OTHR_2034,TPCL_2034,HJI_2034,HH_2035,POP_2035,AEMP_2035,RTL_2035,IND_2035,OTHR_2035,TPCL_2035,HJI_2035,HH_2036,POP_2036,AEMP_2036,RTL_2036,IND_2036,OTHR_2036,TPCL_2036,HJI_2036,HH_2037,POP_2037,AEMP_2037,RTL_2037,IND_2037,OTHR_2037,TPCL_2037,HJI_2037,HH_2038,POP_2038,AEMP_2038,RTL_2038,IND_2038,OTHR_2038,TPCL_2038,HJI_2038,HH_2039,POP_2039,AEMP_2039,RTL_2039,IND_2039,OTHR_2039,TPCL_2039,HJI_2039,HH_2040,POP_2040,AEMP_2040,RTL_2040,IND_2040,OTHR_2040,TPCL_2040,HJI_2040,HH_2041,POP_2041,AEMP_2041,RTL_2041,IND_2041,OTHR_2041,TPCL_2041,HJI_2041,HH_2042,POP_2042,AEMP_2042,RTL_2042,IND_2042,OTHR_2042,TPCL_2042,HJI_2042,HH_2043,POP_2043,AEMP_2043,RTL_2043,IND_2043,OTHR_2043,TPCL_2043,HJI_2043,HH_2044,POP_2044,AEMP_2044,RTL_2044,IND_2044,OTHR_2044,TPCL_2044,HJI_2044,HH_2045,POP_2045,AEMP_2045,RTL_2045,IND_2045,OTHR_2045,TPCL_2045,HJI_2045,HH_2046,POP_2046,AEMP_2046,RTL_2046,IND_2046,OTHR_2046,TPCL_2046,HJI_2046,HH_2047,POP_2047,AEMP_2047,RTL_2047,IND_2047,OTHR_2047,TPCL_2047,HJI_2047,HH_2048,POP_2048,AEMP_2048,RTL_2048,IND_2048,OTHR_2048,TPCL_2048,HJI_2048,HH_2049,POP_2049,AEMP_2049,RTL_2049,IND_2049,OTHR_2049,TPCL_2049,HJI_2049,HH_2050,POP_2050,AEMP_2050,RTL_2050,IND_2050,OTHR_2050,TPCL_2050,HJI_2050
0,791,22,Farmington,None,DAVIS,549.188953,"{'rings': [[[423442.2999999998, 4535297.300000...",1.0,1.959883,80.088889,3.0,18.0,52.0,73.0,74.8,1.0,1.958566,262.069736,3.0,23.0,50.0,76.0,77.8,1.0,1.943265,226.076190,10.0,35.0,173.0,218.0,219.8,1.0,1.927120,250.085177,9.0,35.0,199.0,243.0,244.8,1.0,1.911422,460.032377,12.0,51.0,255.0,318.0,319.8,1.0,1.895471,351.023324,15.0,53.0,275.0,343.0,344.8,1.0,1.879547,369.009479,15.0,53.0,280.0,348.0,349.8,1.0,1.863894,362.026341,15.0,54.0,284.0,353.0,354.8,1.0,1.848839,362.069700,15.0,54.0,284.0,353.0,354.8,1.0,1.834462,363.034483,15.0,54.0,285.0,354.0,355.8,1.0,1.820315,363.034059,15.0,54.0,285.0,354.0,355.8,1.0,1.805903,362.042333,15.0,53.0,285.0,353.0,354.8,1.0,1.790774,362.027933,15.0,52.0,285.0,352.0,353.8,1.0,1.776021,363.009191,15.0,52.0,286.0,353.0,354.8,1.0,1.761581,363.981868,16.0,52.0,286.0,354.0,355.8,1.0,1.747556,365.000000,16.0,52.0,286.0,354.0,355.8,1.0,1.734778,365.019538,16.0,51.0,287.0,354.0,355.8,1.0,1.723698,366.990368,16.0,51.0,289.0,356.0,357.8,1.0,1.713696,368.057391,16.0,51.0,290.0,357.0,358.8,1.0,1.704377,372.030822,16.0,50.0,294.0,360.0,361.8,1.0,1.696044,373.010135,17.0,49.0,295.0,361.0,362.8,1.0,1.688642,374.020033,17.0,49.0,296.0,362.0,363.8,1.0,1.677678,654.021452,18.0,49.0,296.0,363.0,364.8,1.0,1.668817,378.074407,18.0,49.0,298.0,365.0,366.8,1.0,1.660982,382.079224,21.0,49.0,298.0,368.0,369.8,1.0,1.653434,386.055866,23.0,49.0,300.0,372.0,373.8,1.0,1.646608,390.055205,26.0,49.0,301.0,376.0,377.8,1.0,1.640060,399.081839,32.0,49.0,303.0,384.0,385.8,1.0,1.633687,680.054942,50.0,49.

In [7]:
# export
new_se_taz_900 = os.path.join(scratch, '_01_new_se_taz_900')
new_se.spatial.to_featureclass(location=new_se_taz_900,sanitize_columns=False)

'e:\\Projects\\REMM-Process-SE-Output-For-Web\\Outputs\\scratch.gdb\\_01_new_se_taz_900'

### Process Old SE Data

In [8]:
se_dict =  {'TOTHH':'HH',    
            'HHPOP':'POP',
            'RETEMP':'RTL',
            'TOTEMP':'TPCL',
            'ALLEMP':'AEMP',
            'INDEMP':'IND',
            'OTHEMP':'OTHR'}

In [9]:
old_base = taz832_sdf[['CO_TAZID','SHAPE']].copy()
old_base.shape

for csv in old_taz_se:

    df = pd.read_csv(csv)
    category = se_dict[df.iloc[0]['SECategory']]
    val_cols = [col for col in list(df.columns) if 'YEAR' in col and  'D' not in col and int(col[4:]) >= 2019]
    df = df[['CO_TAZID'] + val_cols]
    new_val_cols = [col.replace('YEAR',category + '_') for col in val_cols]
    df.columns = ['CO_TAZID'] + new_val_cols
    old_base = old_base.merge(df, on='CO_TAZID',how='left')

old_se = old_base
old_se.rename({'CO_TAZID': 'COTAZID832'}, axis=1, inplace=True)

In [10]:
# calculate household jobs index
for year in range(2019,2051):
    old_se['HJI_{}'.format(year)] = (old_se['HH_{}'.format(year)] * 1.8) + old_se['TPCL_{}'.format(year)]

In [11]:
old_se.shape

(2881, 258)

In [12]:
# export
old_se_taz_832 = os.path.join(scratch, '_02_old_se_taz_832')
old_se.spatial.to_featureclass(location=old_se_taz_832,sanitize_columns=False)

'e:\\Projects\\REMM-Process-SE-Output-For-Web\\Outputs\\scratch.gdb\\_02_old_se_taz_832'

## Identity

In [13]:
# compare output dimensions
old_se.shape == new_se.shape

False

In [14]:
# merge geometry between taz 832 and output geometries (taz900, )
taz900_identity = arcpy.Identity_analysis(taz900, old_se_taz_832, os.path.join(scratch, '_03_taz832_taz900_identity'))
arcpy.management.DeleteField(taz900_identity,['SA_TAZID', 'DISTMED', 'CityArea', 'OtherArea', 'CO_NAME'], "KEEP_FIELDS")

<Result '.\\Outputs\\scratch.gdb\\_03_taz832_taz900_identity'>

## Apportion the attributes from old geometry to the new ones

1. The Apportion tool still does not work when used with Arcpy
2. Open the Arcgis Pro, ensure the identity layer are present in the map
3. Delete previous apportioning files if present
4. Ensure *dissolve* layer and *identity* layer are present in the contents pane
5. Paste the text from the file, "apportion_command_for_arcgis_pro.txt", in the Arcgis Pro python window to get the apportion output
6. Close Arcgis Pro

In [15]:
os.startfile(r".\REMM-Process-TAZ-Outputs-For-Web.aprx")
os.startfile(r".\apportion_command_for_arcgis_pro_v2.txt")

In [16]:
## uncomment if this tool ever works again from arcpy
# apportion = arcpy.ApportionPolygon_analysis(old_se_taz_832, app_columns, identity, os.path.join(outputs, 'old_se_apportion_to_taz900.shp'), "AREA", "", "", "MAINTAIN_GEOMETRIES") # uncomment 
old_taz_apportion = os.path.join(scratch,'_05_old_se_apportion_to_taz900')

## Dissolve to summarize at geometry

In [17]:
app_columns = list(new_se.columns)[7:]
stat_fields = [[col,'SUM'] for col in app_columns]

#########
# New
#########

# taz (9.0.0)
new_se_taz900_dissolve = arcpy.Dissolve_management(new_se_taz_900, os.path.join(scratch, '_06_new_se_taz900_dissolve'),
                          'SA_TAZID', stat_fields, "MULTI_PART")

# distmed (9.0.0)
new_se_distmed_dissolve = arcpy.Dissolve_management(new_se_taz_900, os.path.join(scratch, '_06_new_se_distmed_dissolve'),
                          'DISTMED', stat_fields, "MULTI_PART")

# city area
new_se_cityarea_dissolve = arcpy.Dissolve_management(new_se_taz_900, os.path.join(scratch, '_06_new_se_cityarea_dissolve'),
                          'CityArea', stat_fields, "MULTI_PART")

# other area
new_se_otherarea_dissolve = arcpy.Dissolve_management(new_se_taz_900, os.path.join(scratch, '_06_new_se_otherarea_dissolve'),
                          'OtherArea', stat_fields, "MULTI_PART")

# county
new_se_county_dissolve = arcpy.Dissolve_management(new_se_taz_900, os.path.join(scratch, '_06_new_se_county_dissolve'),
                          'CO_NAME', stat_fields, "MULTI_PART")



In [19]:
#########
# Old
#########

# taz (9.0.0)
old_se_taz900_dissolve = arcpy.Dissolve_management(old_taz_apportion, os.path.join(scratch, '_06_old_se_taz900_dissolve'),
                          'SA_TAZID', stat_fields, "MULTI_PART")

# distmed (9.0.0)
old_se_distmed_dissolve = arcpy.Dissolve_management(old_taz_apportion, os.path.join(scratch, '_06_old_se_distmed_dissolve'),
                          'DISTMED', stat_fields, "MULTI_PART")

# city area
old_se_cityarea_dissolve = arcpy.Dissolve_management(old_taz_apportion, os.path.join(scratch, '_06_old_se_cityarea_dissolve'),
                          'CityArea', stat_fields, "MULTI_PART")

# other area
old_se_otherarea_dissolve = arcpy.Dissolve_management(old_taz_apportion, os.path.join(scratch, '_06_old_se_otherarea_dissolve'),
                          'OtherArea', stat_fields, "MULTI_PART")

# county
old_se_county_dissolve = arcpy.Dissolve_management(old_taz_apportion, os.path.join(scratch, '_06_old_se_county_dissolve'),
                          'CO_NAME', stat_fields, "MULTI_PART")


In [ ]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [ ]:
# read in new dataframes
new_se_taz900_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_taz900_dissolve[0])
new_se_distmed_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_distmed_dissolve[0])
new_se_cityarea_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_cityarea_dissolve[0])
new_se_otherarea_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_otherarea_dissolve[0])
new_se_county_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_county_dissolve[0])

old_se_taz900_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_taz900_dissolve[0])
old_se_distmed_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_distmed_dissolve[0])
old_se_cityarea_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_cityarea_dissolve[0])
old_se_otherarea_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_otherarea_dissolve[0])
old_se_county_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_county_dissolve[0])

# fill NAs
new_se_taz900_dissolve_df = fill_na_sedf(new_se_taz900_dissolve_df)
new_se_distmed_dissolve_df = fill_na_sedf(new_se_distmed_dissolve_df)
new_se_cityarea_dissolve_df = fill_na_sedf(new_se_cityarea_dissolve_df)
new_se_otherarea_dissolve_df = fill_na_sedf(new_se_otherarea_dissolve_df)
new_se_county_dissolve_df = fill_na_sedf(new_se_county_dissolve_df)

old_se_taz900_dissolve_df = fill_na_sedf(old_se_taz900_dissolve_df)
old_se_distmed_dissolve_df = fill_na_sedf(old_se_distmed_dissolve_df)
old_se_cityarea_dissolve_df = fill_na_sedf(old_se_cityarea_dissolve_df)
old_se_otherarea_dissolve_df = fill_na_sedf(old_se_otherarea_dissolve_df)
old_se_county_dissolve_df = fill_na_sedf(old_se_county_dissolve_df)

# round !! come up with something better
new_se_taz900_dissolve_df = new_se_taz900_dissolve_df.round()
new_se_distmed_dissolve_df = new_se_distmed_dissolve_df.round()
new_se_cityarea_dissolve_df = new_se_cityarea_dissolve_df.round()
new_se_otherarea_dissolve_df = new_se_otherarea_dissolve_df.round()
new_se_county_dissolve_df = new_se_county_dissolve_df.round()

old_se_taz900_dissolve_df = old_se_taz900_dissolve_df.round()
old_se_distmed_dissolve_df = old_se_distmed_dissolve_df.round()
old_se_cityarea_dissolve_df = old_se_cityarea_dissolve_df.round()
old_se_otherarea_dissolve_df = old_se_otherarea_dissolve_df.round()
old_se_county_dissolve_df = old_se_county_dissolve_df.round()

# rename SA_TAZID column
new_se_taz900_dissolve_df.rename({'SA_TAZID':'TAZID'}, axis=1, inplace=True)
old_se_taz900_dissolve_df.rename({'SA_TAZID':'TAZID'}, axis=1, inplace=True)

del new_se_distmed_dissolve_df['SHAPE']
del old_se_distmed_dissolve_df['SHAPE']
del new_se_cityarea_dissolve_df['SHAPE']
del old_se_cityarea_dissolve_df['SHAPE']
del new_se_otherarea_dissolve_df['SHAPE']
del old_se_otherarea_dissolve_df['SHAPE']
del new_se_county_dissolve_df['SHAPE']
del old_se_county_dissolve_df['SHAPE']

In [ ]:
# Remove 'SUM' from column names
to_replace = ['SUM_'+ col for col in app_columns]
replace_dict = dict(zip(to_replace, app_columns))

new_se_taz900_dissolve_df.rename(replace_dict, axis=1, inplace=True)
old_se_taz900_dissolve_df.rename(replace_dict, axis=1, inplace=True)

new_se_distmed_dissolve_df.rename(replace_dict, axis=1, inplace=True)
old_se_distmed_dissolve_df.rename(replace_dict, axis=1, inplace=True)

new_se_cityarea_dissolve_df.rename(replace_dict, axis=1, inplace=True)
old_se_cityarea_dissolve_df.rename(replace_dict, axis=1, inplace=True)

new_se_otherarea_dissolve_df.rename(replace_dict, axis=1, inplace=True)
old_se_otherarea_dissolve_df.rename(replace_dict, axis=1, inplace=True)

new_se_county_dissolve_df.rename(replace_dict, axis=1, inplace=True)
old_se_county_dissolve_df.rename(replace_dict, axis=1, inplace=True)

In [ ]:
# Create folders for deliverables
map_folder = os.path.join(outputs, "map")
if not os.path.exists(map_folder):
    os.makedirs(map_folder)

chart_folder = os.path.join(outputs, "chart")
if not os.path.exists(chart_folder):
    os.makedirs(chart_folder)

In [ ]:
#===============================
# Process TAZ to feature class
#===============================

taz900_sdf.rename({'SA_TAZID':'TAZID'}, axis=1, inplace=True)
taz900_sdf = taz900_sdf[['TAZID', 'CO_NAME', 'DISTMED', 'CityArea', 'OtherArea',  'DEVACRES','SHAPE']].copy()

taz_gdb = os.path.join(map_folder, 'TAZ.gdb')
if not arcpy.Exists(taz_gdb):
    arcpy.CreateFileGDB_management(map_folder, 'TAZ.gdb')


categories = ['HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI']
for c in categories:
    new_cols = [col for col in app_columns if col.split('_')[0] == c]
    new_temp_df = new_se_taz900_dissolve_df[['TAZID'] + new_cols].copy()
    old_temp_df = old_se_taz900_dissolve_df[['TAZID'] + new_cols].copy()
    

    new_new_names = [col.replace(c, 'N') for col in new_cols]
    new_old_names = [col.replace(c, 'O') for col in new_cols]
    new_rename_dict = dict(zip(new_cols, new_new_names))
    old_rename_dict = dict(zip(new_cols, new_old_names))
    new_temp_df.rename(new_rename_dict, axis=1, inplace=True)
    old_temp_df.rename(old_rename_dict, axis=1, inplace=True)


    merged = new_temp_df.merge(old_temp_df, on=['TAZID'], how='left')
    merged = taz900_sdf.merge(merged, on='TAZID', how='left')

    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].fillna(value=0)

    merged.SHAPE.apply(Geometry)
    merged.spatial.set_geometry("SHAPE", inplace=True)
    gsa = GeoSeriesAccessor(merged['SHAPE'])
    merged['ACRES'] = gsa.area * 0.000247105
    merged['ACRES'] = merged['ACRES'].astype(float).round(2)

    outfile = os.path.join(taz_gdb, '{}_PROJECTIONS_by_TAZ'.format(c))
    merged.spatial.to_featureclass(location=outfile, sanitize_columns=False)

    # arcpy.AddField_management(outfile, "ACRES", "FLOAT")
    # exp = "round(!SHAPE.AREA@ACRES!,2)"
    # arcpy.CalculateField_management(outfile, "ACRES", exp, "PYTHON3")

In [ ]:
#=====================================
# Process DISTMED to feature class
#=====================================

distmed_gdb = os.path.join(map_folder, 'DISTMED.gdb')
if not arcpy.Exists(distmed_gdb):
    arcpy.CreateFileGDB_management(map_folder, 'DISTMED.gdb')

categories = ['HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI']
for c in categories:
    new_cols = [col for col in app_columns if col.split('_')[0] == c]
    new_temp_df = new_se_distmed_dissolve_df[['DISTMED'] + new_cols].copy()
    old_temp_df = old_se_distmed_dissolve_df[['DISTMED'] + new_cols].copy()
    

    new_new_names = [col.replace(c, 'N') for col in new_cols]
    new_old_names = [col.replace(c, 'O') for col in new_cols]
    new_rename_dict = dict(zip(new_cols, new_new_names))
    old_rename_dict = dict(zip(new_cols, new_old_names))
    new_temp_df.rename(new_rename_dict, axis=1, inplace=True)
    old_temp_df.rename(old_rename_dict, axis=1, inplace=True)

    merged = new_temp_df.merge(old_temp_df, on='DISTMED', how='left')
    merged = med_districts_sdf.merge(merged, on='DISTMED', how='left')

    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].fillna(value=0)
    
    merged.SHAPE.apply(Geometry)
    merged.spatial.set_geometry("SHAPE", inplace=True)
    gsa = GeoSeriesAccessor(merged['SHAPE'])
    merged['ACRES'] = gsa.area * 0.000247105
    merged['ACRES'] = merged['ACRES'].astype(float).round(2)

    outfile = os.path.join(distmed_gdb, '{}_PROJECTIONS_by_DISTMED'.format(c))
    merged.spatial.to_featureclass(location=outfile, sanitize_columns=False)

    # arcpy.AddField_management(outfile, "ACRES", "FLOAT")
    # exp = "round(!SHAPE.AREA@ACRES!,2)"
    # arcpy.CalculateField_management(outfile, "ACRES", exp, "PYTHON3")


In [ ]:
#====================================
# Process City Area to feature class
#====================================

cityarea_gdb = os.path.join(map_folder, 'CITYAREA.gdb')
if not arcpy.Exists(cityarea_gdb):
    arcpy.CreateFileGDB_management(map_folder, 'CITYAREA.gdb')

categories = ['HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI']
for c in categories:
    new_cols = [col for col in app_columns if col.split('_')[0] == c]
    new_temp_df = new_se_cityarea_dissolve_df[['CityArea'] + new_cols].copy()
    old_temp_df = old_se_cityarea_dissolve_df[['CityArea'] + new_cols].copy()
    

    new_new_names = [col.replace(c, 'N') for col in new_cols]
    new_old_names = [col.replace(c, 'O') for col in new_cols]
    new_rename_dict = dict(zip(new_cols, new_new_names))
    old_rename_dict = dict(zip(new_cols, new_old_names))
    new_temp_df.rename(new_rename_dict, axis=1, inplace=True)
    old_temp_df.rename(old_rename_dict, axis=1, inplace=True)

    merged = new_temp_df.merge(old_temp_df, on='CityArea', how='left')
    merged = city_areas_sdf.merge(merged, on='CityArea', how='left')

    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].fillna(value=0)

    merged.SHAPE.apply(Geometry)
    merged.spatial.set_geometry("SHAPE", inplace=True)
    gsa = GeoSeriesAccessor(merged['SHAPE'])
    merged['ACRES'] = gsa.area * 0.000247105
    merged['ACRES'] = merged['ACRES'].astype(float).round(2)

    outfile = os.path.join(cityarea_gdb, '{}_PROJECTIONS_by_CITYAREA'.format(c))
    merged.spatial.to_featureclass(location=outfile, sanitize_columns=False)

    # arcpy.AddField_management(outfile, "ACRES", "FLOAT")
    # exp = "round(!SHAPE.AREA@ACRES!,2)"
    # arcpy.CalculateField_management(outfile, "ACRES", exp, "PYTHON3")


In [ ]:
#====================================
# Process Other Area to feature class
#====================================

otherarea_gdb = os.path.join(map_folder, 'OTHERAREA.gdb')
if not arcpy.Exists(otherarea_gdb):
    arcpy.CreateFileGDB_management(map_folder, 'OTHERAREA.gdb')

categories = ['HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI']
for c in categories:
    new_cols = [col for col in app_columns if col.split('_')[0] == c]
    new_temp_df = new_se_otherarea_dissolve_df[['OtherArea'] + new_cols].copy()
    old_temp_df = old_se_otherarea_dissolve_df[['OtherArea'] + new_cols].copy()
    

    new_new_names = [col.replace(c, 'N') for col in new_cols]
    new_old_names = [col.replace(c, 'O') for col in new_cols]
    new_rename_dict = dict(zip(new_cols, new_new_names))
    old_rename_dict = dict(zip(new_cols, new_old_names))
    new_temp_df.rename(new_rename_dict, axis=1, inplace=True)
    old_temp_df.rename(old_rename_dict, axis=1, inplace=True)

    merged = new_temp_df.merge(old_temp_df, on='OtherArea', how='left')
    merged = other_areas_sdf.merge(merged, on='OtherArea', how='left')

    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].fillna(value=0)

    merged.SHAPE.apply(Geometry)
    merged.spatial.set_geometry("SHAPE", inplace=True)
    gsa = GeoSeriesAccessor(merged['SHAPE'])
    merged['ACRES'] = gsa.area * 0.000247105
    merged['ACRES'] = merged['ACRES'].astype(float).round(2)

    outfile = os.path.join(otherarea_gdb, '{}_PROJECTIONS_by_OTHERAREA'.format(c))
    merged.spatial.to_featureclass(location=outfile, sanitize_columns=False)

    # arcpy.AddField_management(outfile, "ACRES", "FLOAT")
    # exp = "round(!SHAPE.AREA@ACRES!,2)"
    # arcpy.CalculateField_management(outfile, "ACRES", exp, "PYTHON3")

In [ ]:
#==================================
# Process County to feature class
#==================================

county_gdb = os.path.join(map_folder, 'COUNTY.gdb')
if not arcpy.Exists(county_gdb):
    arcpy.CreateFileGDB_management(map_folder, 'COUNTY.gdb')

categories = ['HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI']
for c in categories:
    new_cols = [col for col in app_columns if col.split('_')[0] == c]
    new_temp_df = new_se_county_dissolve_df[['CO_NAME'] + new_cols].copy()
    old_temp_df = old_se_county_dissolve_df[['CO_NAME'] + new_cols].copy()
    

    new_new_names = [col.replace(c, 'N') for col in new_cols]
    new_old_names = [col.replace(c, 'O') for col in new_cols]
    new_rename_dict = dict(zip(new_cols, new_new_names))
    old_rename_dict = dict(zip(new_cols, new_old_names))
    new_temp_df.rename(new_rename_dict, axis=1, inplace=True)
    old_temp_df.rename(old_rename_dict, axis=1, inplace=True)

    merged = new_temp_df.merge(old_temp_df, on='CO_NAME', how='left')
    merged = counties_sdf.merge(merged, on='CO_NAME', how='left')

    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].fillna(value=0)
    
    merged.SHAPE.apply(Geometry)
    merged.spatial.set_geometry("SHAPE", inplace=True)
    gsa = GeoSeriesAccessor(merged['SHAPE'])
    merged['ACRES'] = gsa.area * 0.000247105
    merged['ACRES'] = merged['ACRES'].astype(float).round(2)


    outfile = os.path.join(county_gdb, '{}_PROJECTIONS_by_COUNTY'.format(c))
    merged.spatial.to_featureclass(location=outfile, sanitize_columns=False)

    # arcpy.AddField_management(outfile, "ACRES", "FLOAT")
    # exp = "round(!SHAPE.AREA@ACRES!,2)"
    # arcpy.CalculateField_management(outfile, "ACRES", exp, "PYTHON3")

In [ ]:
#=======================
# Process TAZ to json
#=======================

chart_taz_folder = os.path.join(chart_folder, 'TAZ')
if not os.path.exists(chart_taz_folder):
    os.makedirs(chart_taz_folder)
else:
    shutil.rmtree(chart_taz_folder)
    os.makedirs(chart_taz_folder)

new_df = new_se_taz900_dissolve_df.copy()
old_df = old_se_taz900_dissolve_df.copy()
id_col = 'TAZID'

categories = ('HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI')
cols = [col for col in new_df.columns if col.startswith(categories)]
new_new_cols = ['NEW_' + col for col in cols]
old_new_cols = ['OLD_' + col for col in cols]

new_rename_dict = dict(zip(cols, new_new_cols))
old_rename_dict = dict(zip(cols, old_new_cols))
new_df.rename(new_rename_dict, axis=1, inplace=True)
old_df.rename(old_rename_dict, axis=1, inplace=True)

del new_df['SHAPE']
del old_df['SHAPE']

df = new_df.merge(old_df, on=id_col, how='left')

for index, row in df.iterrows():
        f = open(os.path.join(chart_taz_folder, "TAZ_{}.json".format(int(row[id_col]))), "a")
        f.write("[\n")
        
        for cat in categories:  
            for year in range(2019,2051):
                
                new_col = '_'.join(['NEW', cat, str(year)])
                old_col = '_'.join(['OLD', cat, str(year)])

                f.write("\t{\n")
                f.write('''\t\t"C":"{}",\n'''.format(cat))
                f.write('''\t\t"Y":{},\n'''.format(year))
                
                f.write('''\t\t"N":{},\n'''.format(row[new_col]))
                f.write('''\t\t"O":{}\n'''.format(row[old_col]))
        
                if cat == "HJI" and year == 2050:
                    f.write("\t}\n")
                else:
                    f.write("\t},\n")

        f.write("]\n")
        f.close()

In [ ]:
#==========================
# Process DISTMED to json
#==========================

chart_DISTMED_folder = os.path.join(chart_folder, 'DISTMED')
if not os.path.exists(chart_DISTMED_folder):
    os.makedirs(chart_DISTMED_folder)
else:
    shutil.rmtree(chart_DISTMED_folder)
    os.makedirs(chart_DISTMED_folder)

new_df = new_se_distmed_dissolve_df.copy()
old_df = old_se_distmed_dissolve_df.copy()
id_col = 'DISTMED'

categories = ('HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI')
cols = [col for col in new_df.columns if col.startswith(categories)]
new_new_cols = ['NEW_' + col for col in cols]
old_new_cols = ['OLD_' + col for col in cols]

new_rename_dict = dict(zip(cols, new_new_cols))
old_rename_dict = dict(zip(cols, old_new_cols))
new_df.rename(new_rename_dict, axis=1, inplace=True)
old_df.rename(old_rename_dict, axis=1, inplace=True)

df = new_df.merge(old_df, on=id_col, how='left')

for index, row in df.iterrows():
        f = open(os.path.join(chart_DISTMED_folder, "{}_{}.json".format(id_col,int(row[id_col]))), "a")
        f.write("[\n")
        
        for cat in categories:  
            for year in range(2019,2051):
                
                new_col = '_'.join(['NEW', cat, str(year)])
                old_col = '_'.join(['OLD', cat, str(year)])

                f.write("\t{\n")
                f.write('''\t\t"C":"{}",\n'''.format(cat))
                f.write('''\t\t"Y":{},\n'''.format(year))
                
                f.write('''\t\t"N":{},\n'''.format(row[new_col]))
                f.write('''\t\t"O":{}\n'''.format(row[old_col]))
        
                if cat == "HJI" and year == 2050:
                    f.write("\t}\n")
                else:
                    f.write("\t},\n")

        f.write("]\n")
        f.close()

In [ ]:
#===========================
# Process CITYAREA to json
#===========================

chart_CITYAREA_folder = os.path.join(chart_folder, 'CITYAREA')
if not os.path.exists(chart_CITYAREA_folder):
    os.makedirs(chart_CITYAREA_folder)
else:
    shutil.rmtree(chart_CITYAREA_folder)
    os.makedirs(chart_CITYAREA_folder)


new_df = new_se_cityarea_dissolve_df.copy()
old_df = old_se_cityarea_dissolve_df.copy()
id_col = 'CityArea'

categories = ('HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI')
cols = [col for col in new_df.columns if col.startswith(categories)]
new_new_cols = ['NEW_' + col for col in cols]
old_new_cols = ['OLD_' + col for col in cols]

new_rename_dict = dict(zip(cols, new_new_cols))
old_rename_dict = dict(zip(cols, old_new_cols))
new_df.rename(new_rename_dict, axis=1, inplace=True)
old_df.rename(old_rename_dict, axis=1, inplace=True)

df = new_df.merge(old_df, on=id_col, how='left')

for index, row in df.iterrows():
        f = open(os.path.join(chart_CITYAREA_folder, "{}_{}.json".format(id_col,row[id_col])), "a")
        f.write("[\n")
        
        for cat in categories:  
            for year in range(2019,2051):
                
                new_col = '_'.join(['NEW', cat, str(year)])
                old_col = '_'.join(['OLD', cat, str(year)])

                f.write("\t{\n")
                f.write('''\t\t"C":"{}",\n'''.format(cat))
                f.write('''\t\t"Y":{},\n'''.format(year))
                
                f.write('''\t\t"N":{},\n'''.format(row[new_col]))
                f.write('''\t\t"O":{}\n'''.format(row[old_col]))
        
                if cat == "HJI" and year == 2050:
                    f.write("\t}\n")
                else:
                    f.write("\t},\n")

        f.write("]\n")
        f.close()

In [ ]:
#===========================
# Process OTHERAREA to json
#===========================

chart_OTHERAREA_folder = os.path.join(chart_folder, 'OTHERAREA')
if not os.path.exists(chart_OTHERAREA_folder):
    os.makedirs(chart_OTHERAREA_folder)
else:
    shutil.rmtree(chart_OTHERAREA_folder)
    os.makedirs(chart_OTHERAREA_folder)


new_df = new_se_otherarea_dissolve_df.copy()
old_df = old_se_otherarea_dissolve_df.copy()
id_col = 'OtherArea'

categories = ('HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI')
cols = [col for col in new_df.columns if col.startswith(categories)]
new_new_cols = ['NEW_' + col for col in cols]
old_new_cols = ['OLD_' + col for col in cols]

new_rename_dict = dict(zip(cols, new_new_cols))
old_rename_dict = dict(zip(cols, old_new_cols))
new_df.rename(new_rename_dict, axis=1, inplace=True)
old_df.rename(old_rename_dict, axis=1, inplace=True)

df = new_df.merge(old_df, on=id_col, how='left')

for index, row in df.iterrows():
        f = open(os.path.join(chart_OTHERAREA_folder, "{}_{}.json".format(id_col,row[id_col])), "a")
        f.write("[\n")
        
        for cat in categories:  
            for year in range(2019,2051):
                
                new_col = '_'.join(['NEW', cat, str(year)])
                old_col = '_'.join(['OLD', cat, str(year)])

                f.write("\t{\n")
                f.write('''\t\t"C":"{}",\n'''.format(cat))
                f.write('''\t\t"Y":{},\n'''.format(year))
                
                f.write('''\t\t"N":{},\n'''.format(row[new_col]))
                f.write('''\t\t"O":{}\n'''.format(row[old_col]))
        
                if cat == "HJI" and year == 2050:
                    f.write("\t}\n")
                else:
                    f.write("\t},\n")

        f.write("]\n")
        f.close()

In [ ]:
#===========================
# Process COUNTY to json
#===========================

chart_COUNTY_folder = os.path.join(chart_folder, 'COUNTY')
if not os.path.exists(chart_COUNTY_folder):
    os.makedirs(chart_COUNTY_folder)
else:
    shutil.rmtree(chart_COUNTY_folder)
    os.makedirs(chart_COUNTY_folder)


new_df = new_se_county_dissolve_df.copy()
old_df = old_se_county_dissolve_df.copy()
id_col = 'CO_NAME'

categories = ('HH', 'POP', 'AEMP', 'RTL', 'IND', 'OTHR', 'TPCL', 'HJI')
cols = [col for col in new_df.columns if col.startswith(categories)]
new_new_cols = ['NEW_' + col for col in cols]
old_new_cols = ['OLD_' + col for col in cols]

new_rename_dict = dict(zip(cols, new_new_cols))
old_rename_dict = dict(zip(cols, old_new_cols))
new_df.rename(new_rename_dict, axis=1, inplace=True)
old_df.rename(old_rename_dict, axis=1, inplace=True)

df = new_df.merge(old_df, on=id_col, how='left')

for index, row in df.iterrows():
        f = open(os.path.join(chart_COUNTY_folder, "{}_{}.json".format(id_col,row[id_col])), "a")
        f.write("[\n")
        
        for cat in categories:  
            for year in range(2019,2051):
                
                new_col = '_'.join(['NEW', cat, str(year)])
                old_col = '_'.join(['OLD', cat, str(year)])

                f.write("\t{\n")
                f.write('''\t\t"C":"{}",\n'''.format(cat))
                f.write('''\t\t"Y":{},\n'''.format(year))
                
                f.write('''\t\t"N":{},\n'''.format(row[new_col]))
                f.write('''\t\t"O":{}\n'''.format(row[old_col]))
        
                if cat == "HJI" and year == 2050:
                    f.write("\t}\n")
                else:
                    f.write("\t},\n")

        f.write("]\n")
        f.close()

In [ ]:
#==============================
# add files to zipped folder
#==============================

d = date.today().strftime("%Y%m%d")

def zipdir(path, ziph, ext=None):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        if ext:
            files = [ fi for fi in files if fi.endswith(ext) ]

        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))



with zipfile.ZipFile(os.path.join(map_folder,'CITYAREA.gdb.zip'),'w', zipfile.ZIP_DEFLATED) as zip_cityarea:
    zipdir(cityarea_gdb, zip_cityarea)

with zipfile.ZipFile(os.path.join(map_folder,'OTHERAREA.gdb.zip'),'w', zipfile.ZIP_DEFLATED) as zip_otherarea:
    zipdir(otherarea_gdb, zip_otherarea)

with zipfile.ZipFile(os.path.join(map_folder,'COUNTY.gdb.zip'),'w', zipfile.ZIP_DEFLATED) as zip_county:
    zipdir(county_gdb, zip_county)

with zipfile.ZipFile(os.path.join(map_folder,'DISTMED.gdb.zip'),'w', zipfile.ZIP_DEFLATED) as zip_distmed:
    zipdir(distmed_gdb, zip_distmed)

with zipfile.ZipFile(os.path.join(map_folder,'TAZ.gdb.zip'),'w', zipfile.ZIP_DEFLATED) as zip_taz:
    zipdir(taz_gdb, zip_taz)


with zipfile.ZipFile(os.path.join('.\\Outputs','Files_For_Web_App_{}.zip'.format(d)),'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir(chart_folder, zipf)
    zipdir(map_folder, zipf, ext='.zip')